# 언어 모델(Language Model)

**단어 시퀀스에 확률을 할당(assign)하는 모델**
- 이전 단어들이 주어졌을 때 다음으로 올 단어에 확률 할당
- 양쪽 단어들로부터 가운데 비어있는 단어에 확률 할당

## 언어 모델링(Language Modeling)

- 주어진 단어들로부터 모르는 단어를 예측하는 일. 

## 단어 시퀀스의 확률 할당
1. 기계 번역(Machine Translation)
 - P(나는 버스를 **탔다**) > P(나는 버스를 **태운다**)
2. 오타 교정(Spell Correction)
 - 선생님이 교실로 부리나케 P(**달려갔다**) > P(**잘려갔다**)
3. 음성 인식(Speech Recognition)
 - P(나는 **메론**을 먹는다) > P(나는 **메롱**을 먹는다)

위처럼 언어 모델은 확률을 통해 보다 적절한 문장을 판단.

## 주어진 이전 단어들로부터 다음 단어 예측하기

1. 단어 시퀀스 확률
$$P(W) = P(w_1, w_2 w_3, w_4, w_5, ..., w_n)$$
2. 다음 단어 등장 확률
$$P(w_n|w_1, ..., w_{n-1})$$
여기서 `|`는 조건부 확률(conditional probability).


전체 단어 시퀀스 W의 확률은 모든 단어가 예측된 후 알 수 있으므로 단어 시퀀스 확률은 다음과 같다.
$$P(W) = P(w_1, w_2, w_3, w_4, w_5, ..., w_n) = \prod^{n}_{i=1}P(w_i|w_1, ..., w_{i-1})$$

위와 같은 다음 단어 확률을 예측하는 모델은 구글과 같은 검색 엔진에서 유용히 사용되고 있다.

# 통계적 언어 모델(SLM, Statistical Language Model)

## 조건부 확률

$$p(B|A) = P(A, B)/P(A)$$
$$P(A, B) = P(A)P(B|A)$$

조건부 확률 연쇄 법칙(chain rule)
$$P(A, B, C, D) = P(A)P(B|A)P(C|A,B)P(D|A,B,C)$$
$$P(x_1, x_2, x_3, ..., x_n) = P(x_1)P(x_2|x_1)P(x_3|x_1, x_2)...P(x_n|x_1, ... x_{n-1})$$

### 조건부 확률 이해에 좋은 예제


In [6]:
import pandas as pd

df = pd.DataFrame({'남학생':[100,80,180],'여학생':[60,120,180], '계':[160,200,360]})
df.index = ['중학생','고등학생','합계']
df

,남학생,여학생,계
중학생,100,60,160
고등학생,80,120,200
합계,180,180,360


위와 같은 표에서 확률을 구해보자.
<br>A = 학생이 남학생인 사건
<br>B = 학생이 여학생인 사건
<br>C = 학생이 중학생인 사건
<br>D = 학생이 고등학생인 사건

1. 학생을 뽑았을 때, 남학생일 확률
 - P(남학생일 확률) / P(학생일 확률) = **P(A)** = 180/360 = 0.5
2. 학생을 뽑았을 때, 고등학생이면서 남학생일 확률
 - P(남학생일 확률 ∩ 고등학생일 확률) / P(학생일 확률) = **P(A∩D)/P(D)** = 80/360 = 0.222...
3. 고등학생 중 한 명을 뽑았을 때, 남학생일 확률 --> 조건부 확률.
 - 고등학생을 뽑았다(첫 번째 사건)<br>뽑은 고등학생이 남학생이었다.(두 번째 사건)
 - P(남학생일 확률 | 고등학생일 확률) = **P(A|D)** = P(남학생일 확률 ∩ 고등학생일 확률) / P(고등학생일 확률) = **P(A∩D)/P(D)** = (80/360) / (200/360) = 80/200 = 0.4

### 문장에 대한 조건부 확률

P(An adorable little boy is spreading smiles)

$$P(An\,adorable\,little\,boy\,is\,spreading\,smiles) = P(An) \times P(adorable|An) \times P(little|An\,adorable) \times P(boy|An\,adorable\,little) \times P(is|An\,adorable\,little\,boy) \times P(spreading|An\,adorable\,little\,boy\,is) \times P(smiles|An\,adorable\,little\,boy\,is\,spreading)$$

### 카운트 기반 접근

기계까 An adorable little boy가 100번 등장했는데 <br>그 다음 is가 등장한 경우가 30번이면 $$P(is|An\,adorable\,littlge\,boy)=30(\%)$$

#### 한계 - 희소 문제(Sparsity Problem)

위처럼 카운트 기반의 모델을 활용하기 위해선 기계가 훈련하는 데이터의 양이 방대해야만 한다.
만약 원하는 문장을 얻으려 할 때 갖고 있는 corpus에서 원하는 문장이 나타난 횟수가 0번이라면 확률은 정의되지 않는다.
<br>이처럼 충분한 데이터를 관측하지 못해 언어를 모델링하지 못하는 문제를 **희소 문제**라고 한다.


# N-gram 

카운트 기반 통계적 접근을 사용하는 SLM의 일종인 언어 모델이다.
<br>그렇지만 앞서 본 언어 모델과는 달리 모든 단어를 고려하는 것이 아닌 n개의 단어를 고려하는 접근 방법을 사용한다.

$$P(is|An\,adorable\,little\,boy) \approx P(is|little\,boy)$$
위처럼 An adorable little boy가 나왔을 때 is가 나올 확류을 구하기 위해 앞 단어 중 임의의 개수만 포함해서 확률을 근사하자.

이 때 임의의 개수를 정하기 위한 기준이 **n-gram**이다.
여기서 n은 n개의 연속적 단어 나열을 의미한다.

* **uni**grams : 1
* **bi**grams : 2
* **tri**grams : 3
* **4**-grams : 4

위에서 든 예시의 경우는 n-1에 해당하는 앞의 단어가 2개이니 n=3인 trigrams이다.

## 한계

1. 희소 문제(Sparsity Problem) -> 여전히 존재.
2. n을 선택하는 것은 trade-off 문제.
  - n을 작게 선택하는 것보다는 커질 수록 성능이 올라감. 하지만 그럴수록 희소 문제 발생.
  - 최대 5를 넘게 잡아서는 안 된다 권장.

# 한국어에서의 언어 모델

한국어는 다음과 같은 이유로 영어보다 언어 모델로 단어 예측하기 훨씬 까다롭다.
1. 한국어는 어순이 중요하지 않음.
 - 아니 진짜 그게 아니라
 - 진짜 아니 그게 아니라
 - 그게 아니라 진짜 아니
 - 아니 그게 아니라 진짜
2. 한국어는 교착어이다.
 - 한국어는 조사가 있다. 이 조사로 굉장히 다양한 형태의 단어가 나온다.
  - '그녀' -> 그녀를, 그녀가, 그녀의, 그녀와, 그녀로
3. 한국어는 띄어쓰기가 제대로 지켜지지 않는다. 띄어쓰기가 이뤄지지 않아도 의미가 전달되기 때문에...
 - 이렇게글을써도읽을수있는우리한글


# Perplexity, PPL(펄플렉서티)

perplexity는 언어 모델의 평가 지표이다.
<br>perplexed는 '헷갈리는'이라는 뜻의 영단어이다.
<br>그래서 여기서 PPL은 '헷갈리는 정도'로 이해하자고 설명하고 있다.
<br>이러한 perplexity는 낮을수록 모델의 성능이 좋다.

$$ PPL(W)=P(w_1, w_2, w_3, ..., w_n)^{-\frac{1}{N}}=\sqrt[\leftroot{-2}\uproot{2}{N}]{\frac{1}{P(w_1, w_2, w_3, ..., w_N}}$$
여기에 체인룰을 적용하면 아래와 같이 된다.

$$ PPL(W)=\sqrt[\leftroot{-2}\uproot{2}{N}]{\frac{1}{P(w_1, w_2, w_3, ..., w_N}}=\sqrt[\leftroot{-2}\uproot{2}{N}]{\frac{1}{\prod^N_{i=1}P(w_i|w_1, w_2, ..., w_{i-1})}}$$

## 분기 계수(Branching factor)

**선택할 수 있는 가능한 경우의 수.**
어떤 언어 모델의 PPL이 10이 나왔다 가정하자. 
<br>그러면 이 언어 모델이 다음 단어를 예측할 때마다 평균 10개의 단어 중 어떤 것이 정답인지 헷갈려했다고 생각할 수 있다.
<br>즉, 이에 대한 분기 계수는 10이라고 할 수 있을 것이다.